In [3]:
from gensim.models import Word2Vec
import time
import numpy as np
from numba import jit
from random import sample,randint
# 套用別人的code，完成詞向量加總平均得到文本向量
from UtilWordEmbedding import MeanEmbeddingVectorizer
import redis

In [4]:
model = Word2Vec.load("./w2v_recipe.model")

In [5]:
doc_vec = np.loadtxt('./doc_vec.csv', delimiter=',')

In [7]:
redis = redis.StrictRedis(host="192.168.1.176", port=6379, decode_responses=True)

In [9]:
mean_embedding_vec = MeanEmbeddingVectorizer(model)

In [11]:
def convert_vector(user_refri_list):
    user_embedding = mean_embedding_vec.transform([user_refri_list])
    return user_embedding.reshape(100, )

In [12]:
def calculate_similarity(user_vec: np.ndarray):
    '''setting function to calculate cosine similarity with numba module'''
    @jit(nopython=True)
    def cosine_similarity_numba(u: np.ndarray, v: np.ndarray):
        assert (u.shape[0] == v.shape[0])
        uv = 0
        uu = 0
        vv = 0
        for i in range(u.shape[0]):
            uv += u[i] * v[i]
            uu += u[i] * u[i]
            vv += v[i] * v[i]
        cos_theta = 1
        if uu != 0 and vv != 0:
            cos_theta = uv / np.sqrt(uu * vv)
            return cos_theta
        else:
            return 0

    '''start calculating'''
    cosine_numba = {}
    for idx, vec in enumerate(doc_vec):
        cosine_numba[idx] = cosine_similarity_numba(user_vec, vec)

    election_list = sorted(cosine_numba.items(), key=lambda item: item[1], reverse=True)
    '''return only top 10 high similarity recipe'''
    return election_list[:10]

In [42]:
def get_recipe_info(user_sim_list):
    for idx,each in enumerate(user_sim_list):
        recipe_id = each[0]
        recipe = redis.hgetall(recipe_id)
        recipe_ing_str = recipe["ingredient"]
        recipe_ing = recipe_ing_str.split(",")
        recipe_like = recipe["like"]
        recipe_ing_set = set([x.split(" ")[0] for x in recipe_ing])
        yield (recipe["recipe"],recipe_like,recipe_ing_str,recipe_ing_set)

In [43]:
def from_set_to_recipe_list(user_set):
    user_vec = convert_vector(user_set)
    user_sim_list = calculate_similarity(user_vec)
    return list(get_recipe_info(user_sim_list))

In [75]:
def refrigerator_cleaner(user_refri_list):
    '''
    user_refri_list: 使用者所擁有的所有食材
    doc_vec: 全食譜的食譜向量(numpy array)
    ----------------------------------------------------------
    此方法為持有食材之使用最大化的食譜推薦系統
    步驟分解
    1. 試圖用全部食材組成一向量
    2. 以此向量計算食譜相似度，得到一個以相似度高低排序過的清單
    3. 將清單內的食譜食材一一和用戶擁有食材比對
    4. 如果有完全符合食材需求的食譜就進行推薦
    5. 若無，則推薦缺漏食材數最少的食譜
    6. 推薦完食譜後扣除使用食材，再以剩餘食材組成一向量，重複步驟2-6
    7. 一直推薦到剩餘食材少於2樣時停止推薦
    '''
    selected_number = 1
    elected_recipe = []
    while True:
        user_set = set(user_refri_list)
        if len(user_set) < 2:
            try:
                print(f'食材不足,只剩下{user_set.pop()},停止提供推薦...')
                break
            except KeyError:
                print(f'食材不足,停止提供推薦...')
                break
        recom_list = from_set_to_recipe_list(user_set)
        print(f"新一輪開始.... 這輪推薦清單有{len(recom_list)}筆")
        print(f"使用者食材庫還有{user_set}")

        waiting_recipe = []  #存放待定食譜
        for idx,each_selected in enumerate(recom_list):
            recipe_set = each_selected[3]
            ing_diff = recipe_set - user_set

            # 判定條件: recipe_set裡面是否有不在user_set內的食材,若無則回傳set(), if判定False
            if not ing_diff:
                print(f"已為您找到第{selected_number}道食譜: {each_selected[0]}")
                print(f'所需食材: {recipe_set}')
                user_refri_list = user_set - recipe_set
                elected_recipe.append(each_selected[0:3]+(set(),))  # 最後一個[]是收藏缺漏食材資訊
                break

            # enter if not full matched
            else:
                if not waiting_recipe:
                    waiting_recipe = each_selected + (ing_diff,)
                    print("產生第一筆待定")
                    print(waiting_recipe[0])
                    continue
                elif len(waiting_recipe[-1]) > len(ing_diff):    # 如果該食譜缺漏食材數更少，就把待定食譜換成此食譜
                    waiting_recipe = each_selected + (ing_diff,)
                    print("產生下一筆待定")
                    print(waiting_recipe[0])
                    continue
                    
            # 拜訪完全部食譜，都沒有完全match的
            if idx+1 == len(recom_list):
                print("進入比對模式")
                print(f"已為您找到第{selected_number}道食譜: {waiting_recipe[0]}")
                print(f'所需食材: {waiting_recipe[3]}, 有缺食材: {waiting_recipe[-1]}')
                user_refri_list = user_set - waiting_recipe[3]
                elected_recipe.append([waiting_recipe[0:3],waiting_recipe[-1]])
                break
        selected_number += 1

    return elected_recipe

In [76]:
user_refri_list = ["麵粉",'鮮奶油','雞蛋','香蕉','洋蔥','蝦','藍苺','豬肉','高麗菜','番茄','大蒜','抹茶',"太白粉"]

In [77]:
first_recom = refrigerator_cleaner(user_refri_list)

新一輪開始.... 這輪推薦清單有10筆
使用者食材庫還有{'蝦', '鮮奶油', '大蒜', '洋蔥', '太白粉', '香蕉', '藍苺', '抹茶', '高麗菜', '豬肉', '麵粉', '雞蛋', '番茄'}
產生第一筆待定
法式鹹派核桃培根派
產生下一筆待定
杏包菇番茄培根鹹蛋糕
已為您找到第1道食譜: 焦糖香蕉蛋糕七吋咕咕霍
所需食材: {'鮮奶油', '香蕉', '藍苺', '麵粉', '雞蛋'}
新一輪開始.... 這輪推薦清單有10筆
使用者食材庫還有{'蝦', '大蒜', '洋蔥', '太白粉', '抹茶', '高麗菜', '豬肉', '番茄'}
已為您找到第2道食譜: 明蝦套餐小確幸
所需食材: {'蝦', '大蒜', '洋蔥', '高麗菜', '豬肉'}
新一輪開始.... 這輪推薦清單有10筆
使用者食材庫還有{'太白粉', '抹茶', '番茄'}
產生第一筆待定
馬鈴薯手搓甜湯圓
進入比對模式
已為您找到第3道食譜: 馬鈴薯手搓甜湯圓
所需食材: {'太白粉', '馬鈴薯', '抹茶'}, 有缺食材: {'馬鈴薯'}
食材不足,只剩下番茄,停止提供推薦...


In [78]:
first_recom

[('焦糖香蕉蛋糕七吋咕咕霍',
  '146',
  '鮮奶油 68 gram,麵粉 150 gram,雞蛋 180 gram,香蕉 135 gram,藍苺 140 gram',
  set()),
 ('明蝦套餐小確幸',
  '44',
  '蝦 260 gram,洋蔥 80 gram,高麗菜 210 gram,豬肉 270 gram,大蒜 40 gram',
  set()),
 [('馬鈴薯手搓甜湯圓', '208', '馬鈴薯 250 gram,太白粉 100 gram,抹茶 10 gram'), {'馬鈴薯'}]]

In [84]:
def quick_recommend(user_refri_list):
    user_set = set(user_refri_list)
    random_set = sample(user_refri_list, randint(2, len(user_refri_list)))
    recom_list = from_set_to_recipe_list(random_set)

    waiting_recipe = []  # 存放待定食譜
    for idx, each_selected in enumerate(recom_list):
        recipe_set = each_selected[3]
        ing_diff = recipe_set - user_set

        if not ing_diff:
            return each_selected[0:3]+(set(),)

        else:
            if not waiting_recipe:
                waiting_recipe = each_selected + (ing_diff,)
                continue
            elif len(waiting_recipe[-1]) > len(ing_diff):  # 如果該食譜缺漏食材數更少，就把待定食譜換成此食譜
                waiting_recipe = each_selected + (ing_diff,)
                continue

    return waiting_recipe[0:3] + (waiting_recipe[-1],)

In [85]:
def recipe_recommend_system(user_refri_list,recipe_number=5):
    '''
    user_refri_list: 使用者所擁有的所有食材
    doc_vec: 全食譜的食譜向量(numpy array)
    ----------------------------------------------------------
    recipe_number: 期望推薦的食譜數量，預設為10道
    此方法為從持有食材隨機抽樣進行推薦的系統
    步驟分解
    1. 隨機選出不定數量的食材組合成一向量
    2. 以此向量計算食譜相似度，得到一個以相似度高低排序過的清單
    3. 將清單內的食譜食材一一和用戶擁有食材比對
    4. 如果有完全符合食材需求的食譜就推薦出來，若無就pass
    5. 該清單比對完後進行下一輪，重複步驟1-5
    6. 若連續三輪都找不到完全符合食材需求的食譜，則從次輪開始進入快速推薦
    6-1. 快速推薦: 每一輪必推薦一道食譜，如果有完全符合的則優先，若無則推薦食材缺少數最低的食譜
    7. 直到推薦清單達到設定值後便停止推薦
    '''

    selected_number = 1
    not_found_count = 0
    recommend_recipe_name = set()
    elected_recipe = []
    while True:
        if selected_number == recipe_number + 1:
            print(f'已完成{recipe_number}道食譜推薦...')
            break

        user_set = set(user_refri_list)
        random_set = sample(user_refri_list,randint(2,len(user_refri_list)))
        recom_list = from_set_to_recipe_list(random_set)


        for idx,each_selected in enumerate(recom_list):
            recipe_set = each_selected[3]
            ing_diff = recipe_set - user_set
            if each_selected[0] in recommend_recipe_name:
                continue

            # 判定條件: recipe_set裡面是否有不在user_set內的食材,若無則回傳set(), if判定False
            if not ing_diff:
                print(f"已為您找到第{selected_number}道食譜: {each_selected[0]}")
                print(f'所需食材: {recipe_set}')
                selected_number += 1
                not_found_count = 0
                recommend_recipe_name.add(each_selected[0])
                elected_recipe.append(each_selected[0:3]+(set(),))
                break
            else:
                pass

            if idx+1 == len(recom_list):
                not_found_count += 1


        # 判斷是否進入快速推薦
        if not_found_count >= 3:
            print('連續3次無法推薦完全符合的食譜，進入第二階段快速推薦')
            for i in range(recipe_number-selected_number+1):
                #　持續找，直到找到沒有重複的食譜
                while True:
                    quick_recipe = quick_recommend(user_refri_list)
                    if quick_recipe[0] not in recommend_recipe_name:
                        break

                elected_recipe.append(quick_recipe)

                selected_number += 1
                recommend_recipe_name.add(quick_recipe[0])

            break

        else:
            continue

    return elected_recipe

In [90]:
second_recom = recipe_recommend_system(user_refri_list)

已為您找到第1道食譜: 藍莓雪糕
所需食材: {'香蕉', '藍苺'}
已為您找到第2道食譜: 焦糖香蕉蛋糕七吋咕咕霍
所需食材: {'鮮奶油', '香蕉', '藍苺', '麵粉', '雞蛋'}
連續3次無法推薦完全符合的食譜，進入第二階段快速推薦


In [91]:
second_recom

[('藍莓雪糕', '308', '香蕉 150 gram,藍苺 140 gram', set()),
 ('焦糖香蕉蛋糕七吋咕咕霍',
  '146',
  '鮮奶油 68 gram,麵粉 150 gram,雞蛋 180 gram,香蕉 135 gram,藍苺 140 gram',
  set()),
 ('藍莓果昔',
  '22',
  '蔬菜 90.0 ml,雞蛋 90.0 ml,綜合水果 120 gram,香蕉 180 ml,綜合核果 70 gram,巧克力 70 gram',
  {'巧克力', '綜合核果', '綜合水果', '蔬菜'}),
 ('萬家香大吟釀薄鹽煎餃佐特製五味醬', '85', '麵粉 180 gram,大蒜 40 gram', set()),
 ('綠藤芽花生醬香蕉精力湯', '93', '高麗菜 30 gram,香蕉 150 gram', set())]

In [92]:
a = {'ingredient': '綜合核果 300 gram,麥芽 120 gram,雞蛋 60 gram,蔓越莓 50 gram',
                         'recipe': '過年小零嘴牛軋糖',
                         'recipe_id': '123',
                         'seasoning': '砂糖 170 gram,蜂蜜 50 gram,水 60 ml',
                         'cluster': 'cluster6',
                         'image': 'https://imageproxy.icook.network/resize?height=600&nocrop=false&stripmeta=true&type=auto&url=http%3A%2F%2Ftokyo-kitchen.icook.tw.s3.amazonaws.com%2Fuploads%2Frecipe%2Fcover%2F99211%2F0f2ce1865bd45541.jpg&width=800',
                         'like': '719',
                         'url': 'https://icook.tw/recipes/99211',
                         'time': '30分鐘',
                         'quantity': '1'}

In [97]:
aa = set(x.split(" ")[0] for x in a["ingredient"].split(","))

In [126]:
",".join(aa)

'蔓越莓,雞蛋,麥芽,綜合核果'

In [100]:
b = set(['蔓越莓', '雞蛋'])

In [110]:
import json

In [112]:
recipe_menu_tmp = json.load(open("../素材/recipe_reply/reply.json", encoding='utf-8'))

In [115]:
cc = recipe_menu_tmp['contents']

In [129]:
cc[0]["footer"]["contents"][1]

{'type': 'button',
 'action': {'type': 'postback',
  'label': '我要這個食譜',
  'text': '您已成功選入食譜，將幫您從智能冰箱中扣除所需食材',
  'data': 'confirm='},
 'color': '#39D230FF',
 'style': 'primary'}

In [130]:
redis.hgetall("1")

{'recipe': '草莓棒棒糖',
 'ingredient': '草莓 260 gram,牛奶巧克力 110 gram,巧克力 70 gram,煉乳 60 gram',
 'seasoning': '水 50 ml,味醂 50 ml,鰹魚粉 10 gram',
 'recipe_id': '1',
 'image': 'https://imageproxy.icook.network/resize?height=600&nocrop=false&stripmeta=true&type=auto&url=http%3A%2F%2Ftokyo-kitchen.icook.tw.s3.amazonaws.com%2Fuploads%2Frecipe%2Fcover%2F99004%2F12cd6f5bc1ab00ab.jpg&width=800',
 'quantity': '2',
 'like': '121',
 'cluster': 'cluster6',
 'url': 'https://icook.tw/recipes/99004',
 'time': '20分鐘'}

In [132]:
key = redis.keys()

In [133]:
len(key)

180198

In [140]:
redis.srandmember("userid")

'U429ec102b46a5332b32c4f1a8b3b04db'

In [164]:
k = redis.hgetall("U429ec102b46a5332b32c4f1a8b3b04db")

In [162]:
redis.hdel('U429ec102b46a5332b32c4f1a8b3b04db','芭樂')

1

In [167]:
del k["雞蛋"]

In [168]:
k

{'番茄': '10,顆,2020/10/24',
 '羊肉': '7,個,2020/10/21',
 '大蒜': '2,根,2020/10/23',
 '蔥': '5,個,2020/10/24',
 '豬肉': '200,gram,2020/10/19',
 '高麗菜': '1,顆,2020/10/23',
 '金針菇': '2,根,2020/10/23',
 '太白粉': '500,gram,2020/10/01'}